#В этом файле работы загрузим данные в модель наименьших квадратов, и посмотрим что и как у нас получится

In [96]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error
from scipy.stats import ttest_1samp
from scipy.stats import shapiro, levene, f_oneway, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp


In [97]:
df=pd.read_excel("done_data.xlsx")

## Непосредственно модель

In [98]:
df= df.drop(columns=["Unnamed: 0", "Фамилия Имя"])
X = df.drop(columns="Стоимость")
X = sm.add_constant(X)
y = df["Стоимость"]
model = sm.OLS(y, X)
result = model.fit()
print(result.summary())
MAE=mean_absolute_error(y, result.predict(X))
RMSE=np.sqrt(mean_squared_error(y, result.predict(X)))
print(MAE)
print(RMSE)
latex_summary = result.summary().as_latex()
with open('summary_1.tex', 'w') as f:
    f.write(latex_summary)


                            OLS Regression Results                            
Dep. Variable:              Стоимость   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                     4.244
Date:                Thu, 05 Dec 2024   Prob (F-statistic):           4.41e-18
Time:                        22:40:37   Log-Likelihood:                 378.38
No. Observations:                 297   AIC:                            -488.8
Df Residuals:                     163   BIC:                             6.203
Df Model:                         133                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

# Проверим гипотезы

### Первая гипотеза о переоценнености игроков АПЛ

In [99]:
X_epl = X[df["Лига_АПЛ"] == 1]
y_epl = y[df["Лига_АПЛ"] == 1]
predicted_value = result.predict(X_epl)
actual_value_epl = df["Стоимость"][df["Лига_АПЛ"] == 1]
overvaluation = actual_value_epl - predicted_value

print("Среднее overvaluation:", np.mean(overvaluation))

t_stat, p_value = ttest_1samp(overvaluation, 0)

print("Результаты t-теста по игрокам АПЛ:")
print("t-статистика:", t_stat)
print("p-значение:", p_value)

alpha = 0.05
if p_value < alpha:
    if np.mean(overvaluation) > 0:
        print("Отвергаем H0: игроки АПЛ переоценены.")
    else:
        print("Отвергаем H0: игроки АПЛ недооценены (фактическая стоимость меньше предсказанной).")
else:
    print("Нет статистически значимых доказательств переоценки или недооценки игроков АПЛ.")

Среднее overvaluation: -3.4871854777230013e-16
Результаты t-теста по игрокам АПЛ:
t-статистика: -5.571784027324558e-14
p-значение: 0.9999999999999556
Нет статистически значимых доказательств переоценки или недооценки игроков АПЛ.


Первая гипотеза отвергается не отвергается и мы не можем сказать, что игроки АПЛ переоценены

### Проверим вторую гипотезу о том , что атакующие более ценный товар, чем защитники и полузатники

In [100]:
position_columns = [col for col in df.columns if col.startswith('Позиция на поле_')]
attack_positions = [
    'Позиция на поле_Левый Вингер',
    'Позиция на поле_Правый Вингер',
    'Позиция на поле_Оттянутый нап.',
    'Позиция на поле_Центральный нап.'
]

midfield_positions = [
    'Позиция на поле_Опорный полузащитник',
    'Позиция на поле_Правый полузащитник',
    'Позиция на поле_Центр. полузащитник'
]

defence_positions = [
    'Позиция на поле_Левый защитник',
    'Позиция на поле_Правый защитник',
    'Позиция на поле_Центр. защитник'
]

def get_player_position(row):
    for c in position_columns:
        if row[c] == 1:
            return c
    return 'Unknown'
def categorize_position(pos):
    if pos in attack_positions:
        return 'Attack'
    elif pos in midfield_positions:
        return 'Midfield'
    elif pos in defence_positions:
        return 'Defence'
    else:
        return 'Other'
df['position'] = df.apply(get_player_position, axis=1)
df['position_group'] = df['position'].apply(categorize_position)

attack_values = df[df['position_group'] == 'Attack']['Стоимость'].dropna()
defence_values = df[df['position_group'] == 'Defence']['Стоимость'].dropna()
midfield_values = df[df['position_group'] == 'Midfield']['Стоимость'].dropna()
p_kruskal = kruskal(attack_values, defence_values, midfield_values)[1]
print("Kruskal-Wallis p-value:", p_kruskal)
if p_kruskal < 0.05:
    print("Есть статистически значимые различия между группами позиций.")
else:
    print("Нет статистически значимых различий между группами позиций.")
data = pd.concat([attack_values, defence_values, midfield_values], axis=0, keys=['Attack', 'Defence', 'Midfield'])
df_for_posthoc = pd.DataFrame({'values': data.values, 'groups': data.index.get_level_values(0)})
posthoc = sp.posthoc_dunn(df_for_posthoc, val_col='values', group_col='groups', p_adjust='bonferroni')
mean_att=df[df['position_group'] == 'Attack']['Стоимость'].mean()
mean_def=df[df['position_group'] == 'Defence']['Стоимость'].mean()
mean_mid=df[df['position_group'] == 'Midfield']['Стоимость'].mean()
print(posthoc,mean_att,mean_mid,mean_def)


Kruskal-Wallis p-value: 0.034758281225469756
Есть статистически значимые различия между группами позиций.
            Attack   Defence  Midfield
Attack    1.000000  0.034012  0.323787
Defence   0.034012  1.000000  1.000000
Midfield  0.323787  1.000000  1.000000 0.15544768069039916 0.12285714285714287 0.09402628434886501
